In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from pathlib import Path
import os

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Linear Regression

# Data Preprocesing Process

# Training Datasets
train_dataset = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
X = train_dataset.iloc[0:, 1:-1].values
y = train_dataset['Pawpularity'].values

print(X)


# Training Datasets
test_dataset = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
test_X = test_dataset.iloc[0:, 1:].values
test_Id = test_dataset.iloc[0:, 0:1].values

print(test_Id)

In [ ]:
# Image Regression

base_dir = Path('/kaggle/input')

filepaths = pd.Series(list(base_dir.glob(r'**/*.jpg')), name='Filepath').astype(str)

animal_types = pd.Series(filepaths.apply(lambda x: os.path.split(os.path.split(x)[0])[1]), name='Animal Type').astype(np.str)

for idx, value in enumerate(animal_types):
    if(value == 'test'):
        animal_types[idx] = 1.0
    elif(value == 'train'):
        animal_types[idx] = 2.0

images = pd.concat([filepaths, animal_types], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

print(images)

In [ ]:
# Linear Regression

# Data Preprocesing Proces 
# We could train and test by calculate a standardisation, normalization value from scratch. In contrast, this code use imported library instead.
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
# Training a data
# Below line is a vector of predicted value from X_train which is Salary
regressor.fit(X, y)

In [ ]:
# Linear Regression

# y_pred is a vector of predicted value from X_test which is Salary
y_pred = regressor.predict(test_X)

print('y_pred', y_pred)

In [ ]:
# Image Regression

image_df = images.sample(5000, random_state=1).reset_index(drop=True)

train_df, test_df = train_test_split(images, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
# Image Regression

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
# Linear Regression

import csv

header = ['Id', 'Pawpularity']
data = []

for index in range(len(y_pred)): 
    data.append([test_Id[index, 0], y_pred[index]])
    
with open('/kaggle/working/submission.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

In [ ]:
# Image Regression
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Animal Type',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Animal Type',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Animal Type',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

In [ ]:
# new_train_images = np.asarray(train_images).astype(np.float32)
# new_val_images = np.asarray(val_images).astype(np.float32)

inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

# history = model.fit(
#     new_train_images,
#     validation_data=new_val_images,
#     epochs=1,
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(
#             monitor='val_loss',
#             patience=5,
#             restore_best_weights=True
#         )
#     ]
# )